### Test sentiment ML 2

In [1]:
from parse import *
from word_tools import *


In [2]:
from word_tools import make_bagofwords

nb_words = 10000

bow = make_bagofwords("corpus_ready.data", nb_words)
print(bow[-50:])

['cette', 'jean', 'contre', 'être', 'lettre', 'tous', 'européens', 'même', 'bien', 'politique', 'français', 'ils', 'tout', 'faire', 'comme', 'lui', 'jaunes', 'président', 'sont', 'gilets', 'nous', 'raffarin', 'mais', 'fait', 'renaissance', 'tribune', 'européennes', 'par', 'son', 'france', 'via', 'plus', 'avec', 'sur', 'européenne', 'europe', 'aux', 'vous', 'dans', 'emmanuel', 'qui', 'pas', 'que', 'une', 'des', 'pour', 'est', 'les', '', 'macron']


In [3]:
from word_tools import vectorize_tweets
vect = vectorize_tweets("tw_db_prepared.data", bow)

In [4]:
import numpy as np
keys = [k for k in vect]
print(np.shape(vect[keys[0]]['vectorized']))

(45,)


In [5]:
print(np.max([np.shape(vect[k]['vectorized'])[0] for k in vect]))

45


In [6]:


def extract_data(vectsdict):
    import numpy as np
    vects = []
    labels = []
    
    for key in vectsdict :
        if not np.all(vectsdict[key]['label'] == 0) :
            vects.append(vectsdict[key]['vectorized'])
            labels.append(vectsdict[key]['label'])
    return np.array(vects), np.array(labels)

def decode_output(out_array) :
    if out_array[0] == np.max(out_array) :
        return "irr", np.max(out_array)
    elif out_array[1] == np.max(out_array) :
        return "neg", np.max(out_array)
    elif out_array[2] == np.max(out_array) :
        return "neu",np.max(out_array)
    elif out_array[3] == np.max(out_array) :
        return "pos",np.max(out_array)
    else :
        return "err",0
    
data, labels = extract_data(vect)


percentage_train = 0.8
borne = int(percentage_train*len(data))
X_train = data[:borne]
Y_train = labels[:borne]
X_test = data[borne:]
Y_test = labels[borne:]

print(X_train[0])
input_l = len(X_train[0])

[8700    0 8086 8601 8705 8603 1643 8696    0 7388 8250 8689 8700 8675
 8672 8696 8244 1979 8630 7623 8700 5841 8696 8703    0 7346 8700 8577
 7245 8701 8703 3927 8403 1735 8241 8704    0    0    0    0    0    0
    0    0    0]


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, SpatialDropout1D
from keras.layers.embeddings import Embedding

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(nb_words, embed_dim,input_length =input_l))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())
# compile the model

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# print(model.summary())
batch = 100
model.fit(X_train, Y_train, epochs=80, batch_size=batch, verbose=1)

Epoch 1/80
806/806 [==============================] - 2s 3ms/step - loss: 1.2746 - acc: 0.4913
Epoch 2/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1486 - acc: 0.5285
Epoch 3/80
806/806 [==============================] - 2s 2ms/step - loss: 1.1497 - acc: 0.5285
Epoch 4/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1587 - acc: 0.5285
Epoch 5/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1454 - acc: 0.5285
Epoch 6/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1433 - acc: 0.5285
Epoch 7/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1493 - acc: 0.5285
Epoch 8/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1522 - acc: 0.5285
Epoch 9/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1518 - acc: 0.5285
Epoch 10/80
806/806 [==============================] - 1s 2ms/step - loss: 1.1438 - acc: 0.5285
Epoch 11/80
806/806 [============================

In [15]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
# loss, accuracy = model.evaluate(X_train, Y_train)
print(accuracy)

0.5297029705921023


In [9]:
test = "Aux chiottes macron tu as fais du mal aux gilets jaunes"

test_vect = vectorize_tweet(test, "pos", bow, input_l)
v = np.array(test_vect['vectorized'])
v = v.reshape((1,input_l))

res = model.predict([v])[0]
out = decode_output(res)
print("tweet is evaluated " + out[0])

tweet is evaluated neg


In [10]:
from parse import sample_data
testset = sample_data("tw_db_prepared.data", 10)
print(testset)

['revoir les taux votes par csp niveau revenu lors des législatives tant que les classesmoyennes sup sont pas lessivées comme les gilets jaunes par macron président des ultras riches lrem sera devant attendre ans désindexation retraite ', 'vous êtes une grande naïveté monsieur ensorcelé par gourou soudaine empathie pour ceux qui sont rien vous surprend pas cette période approchant les élections européennes macron essaie soigner son image arrogant hors sol ', 'macron fera jamais rien qui puisse choquer les musulmans les islamistes car est une part importante son électorat trahison ', 'dictature régime politique dans lequel tous les pouvoirs sont entre les mains une seule personne oui effectivement macron est pas seul ceux qui ont intérêt que reste comme aide bcp vrai décide pas chose ', 'affaire macron benalla réponse nos détracteurs via ', 'est macron qui est complice pire', 'bref jsuis vener cause vos histoires plus macron fait des bottle flip pendant que samedi peux plus sortir chez 

In [11]:
for tw in testset :
    vec = vectorize_tweet(tw,'???', bow, input_l)
    print(vec['text'])
    res = model.predict(vec['vectorized'].reshape((1,input_l)))[0]
    out = decode_output(res)
    print("tweet is evaluated " + out[0] + "\n\n")

revoir les taux votes par csp niveau revenu lors des législatives tant que les classesmoyennes sup sont pas lessivées comme les gilets jaunes par macron président des ultras riches lrem sera devant attendre ans désindexation retraite 
tweet is evaluated neg


vous êtes une grande naïveté monsieur ensorcelé par gourou soudaine empathie pour ceux qui sont rien vous surprend pas cette période approchant les élections européennes macron essaie soigner son image arrogant hors sol 
tweet is evaluated neg


macron fera jamais rien qui puisse choquer les musulmans les islamistes car est une part importante son électorat trahison 
tweet is evaluated neg


dictature régime politique dans lequel tous les pouvoirs sont entre les mains une seule personne oui effectivement macron est pas seul ceux qui ont intérêt que reste comme aide bcp vrai décide pas chose 
tweet is evaluated neg


affaire macron benalla réponse nos détracteurs via 
tweet is evaluated neu


est macron qui est complice pire
tweet 